In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score
import datetime
import os
from Imputer.knn_imputer import KNNImputer
import pdb
imagedir = 'images'
if not os.path.isdir(imagedir):
    os.mkdir(imagedir)

# For numeric stability
EPSILON = 1e-10

df = pd.read_csv(
    'houselistings_simulated.csv', parse_dates=['ListingDate', 'SalesDate'])

dataset_end = df['SalesDate'].max()

FIRST_CUTOFF = pd.to_datetime('2017-01-01')
LAST_CUTOFF = df['SalesDate'].max() - pd.DateOffset(months=1)


def split_at(df, start_date=None, end_date=None):
    data_idx = np.ones(df.shape[0], dtype=bool)
    if start_date is not None:
        data_idx = data_idx & (df.SalesDate >= start_date)
    if end_date is not None:
        data_idx = data_idx & (df.ListingDate < end_date)
    return df.loc[data_idx, :].copy()


def scale(df, scaling_mean=None, scaling_std=None):
    numeric_features_train = df.select_dtypes(include=[np.number]).copy()
    if scaling_mean is None:
        scaling_mean, scaling_std = numeric_features_train.mean(
        ), numeric_features_train.std()
    numeric_features_train = (
        numeric_features_train - scaling_mean) / scaling_std
    return numeric_features_train, scaling_mean, scaling_std


def build_imputer(numeric_features_train):
    imputer = KNNImputer()
    imputer.fit(numeric_features_train)
    return imputer


def apply_imputer(imputer, features):
    features = features.copy()
    imputer.fill_in(features)
    return features


def build_Y(df, end):
    sold = df['SalesDate'] < end
    Y = df['SalesDate'].clip(upper=end).fillna(end) - df['ListingDate']
    return sold, Y.dt.days.astype(float)


class Model():
    def __init__(self, input_size, layer_sizes):
        self.input_size = input_size
        self.layer_sizes = layer_sizes
        smoothing_factor = 1

        self.sold = tf.placeholder(tf.float32, shape=(None))
        self.x = tf.placeholder(tf.float32, shape=(None, input_size))
        self.y = tf.placeholder(tf.float32, shape=(None))

        self.layers = [self.x]
        for layer_size in layer_sizes:
            next_layer = tf.nn.leaky_relu(
                tf.layers.dense(self.layers[-1], layer_size))
            self.layers.append(next_layer)

        self.output = tf.nn.softplus(tf.layers.dense(self.layers[-1], 1))[:, 0]

        self.loss_indicator = (tf.cast(self.output < self.y, tf.float32) *
                               (1 - self.sold) + self.sold)

        def build_loss(loss, loss_indicator):
            loss_numerator = tf.reduce_sum(loss * self.loss_indicator)
            loss_denominator = (tf.reduce_sum(self.loss_indicator)) + EPSILON
            return loss_numerator / loss_denominator

        error_by_sample = (self.output - self.y)
        percent_error_by_sample = error_by_sample / (self.y + smoothing_factor)

        MSPE_raw = tf.reduce_mean(tf.square(percent_error_by_sample))
        MSE_raw = tf.reduce_mean(tf.square(error_by_sample))

        # Unused, but a demonstration of reasonable loss functions:
        self.MSPE = build_loss(MSPE_raw, self.loss_indicator)
        self.MSE = build_loss(MSE_raw, self.loss_indicator)

        # loss function used that takes into account both raw and percent loss
        MSPE_MSE_geometric_mean = tf.sqrt(MSPE_raw * MSE_raw)
        self.loss = build_loss(MSE_raw, self.loss_indicator)

        MAPE_raw = tf.reduce_mean(tf.abs(percent_error_by_sample))
        self.MAPE = build_loss(MAPE_raw, self.loss_indicator)

        opt_fcn = tf.train.AdamOptimizer()

        def apply_clipped_optimizer(opt_fcn,
                                    loss,
                                    clip_norm=.1,
                                    clip_single=.03,
                                    clip_global_norm=False):
            gvs = opt_fcn.compute_gradients(loss)

            if clip_global_norm:
                gs, vs = zip(*[(g, v) for g, v in gvs if g is not None])
                capped_gs, grad_norm_total = tf.clip_by_global_norm(
                    [g for g in gs], clip_norm)
                capped_gvs = list(zip(capped_gs, vs))
            else:
                grad_norm_total = tf.sqrt(
                    tf.reduce_sum([
                        tf.reduce_sum(tf.square(grad)) for grad, var in gvs
                        if grad is not None
                    ]))
                capped_gvs = [(tf.clip_by_value(grad, -1 * clip_single,
                                                clip_single), var)
                              for grad, var in gvs if grad is not None]
                capped_gvs = [(tf.clip_by_norm(grad, clip_norm), var)
                              for grad, var in capped_gvs if grad is not None]

            optimizer = opt_fcn.apply_gradients(
                capped_gvs, global_step=tf.train.get_global_step())

            return optimizer, grad_norm_total

        self.optimizer, self.grad_norm_total = apply_clipped_optimizer(
            opt_fcn, self.loss)

        self.sess = tf.Session()
        self.sess.run(tf.global_variables_initializer())

        self.trn_losses = []
        self.val_losses = []
        self.r2_scores = []

    def train_one_epoch(self, X_train, Y_train, sold_train, bs):
        # Train an epoch
        trn_loss = []
        # Randomly shuffle data and prepare for training
        trn_samples = X_train.shape[0]
        order = np.arange(trn_samples)
        np.random.shuffle(order)
        num_batches = (trn_samples // bs) + 1
        for itr in range(trn_samples // bs):
            rows = order[itr * bs:(itr + 1) * bs]
            if itr + 1 == num_batches:
                rows = order[itr * bs:]
            X_active, Y_active, Sold_active = [
                mat[rows] for mat in [X_train, Y_train, sold_train]
            ]
            feed_dict = {
                self.x: X_active,
                self.y: Y_active,
                self.sold: Sold_active
            }
            _, loss = self.sess.run([self.optimizer, self.loss], feed_dict)
            trn_loss.append(loss)
        self.trn_losses.append(np.mean(trn_loss))

    def validate(self, X_test, Y_test, sold_test, bs=512):
        val_samples = X_test.shape[0]
        num_batches = val_samples // bs
        order = np.arange(val_samples)
        val_loss_total = 0
        yhat_all = np.zeros(0)
        for itr in range(num_batches):
            rows = order[itr * bs:(itr + 1) * bs]
            if itr + 1 == num_batches:
                rows = order[itr * bs:]
            X_active, Y_active, Sold_active = [
                mat[rows] for mat in [X_test, Y_test, sold_test]
            ]
            feed_dict = {
                self.x: X_active,
                self.y: Y_active,
                self.sold: Sold_active
            }
            val_loss, yhat = self.sess.run([self.loss, self.output], feed_dict)
            yhat_all = np.concatenate((yhat_all, yhat), 0)
            val_loss_total += (val_loss * Sold_active.sum())

        val_loss_avg = val_loss_total / sold_test.sum()

        self.val_losses.append(val_loss_avg)
        self.r2_scores.append(r2_score(Y_test, yhat_all))
        return yhat_all

    def train(self, Xtrn, Xval, Ytrn, Yval, Soldtrn, Soldval, epochs, bs=512):
        # Everything is set. Now train and validate
        for epoch in range(epochs):
            # run one epoch train and validation
            self.train_one_epoch(Xtrn, Ytrn, Soldtrn, bs)
            yhat = self.validate(Xval, Yval, Soldval)

            if (epoch % 3 == 0) or (epoch == epochs - 1):
                # Occasionally print to command line to inspect performance
                print('epoch:', epoch, 'train loss: ', self.trn_losses[-1],
                      'val loss: ', self.val_losses[-1], 'r2_score:',
                      self.r2_scores[-1])
        return yhat

    def visualize(self, name, fname=None):
        # Visualize training and validation losses and r2 scores on one plot
        _, ax1 = plt.subplots()
        ax2 = ax1.twinx()
        ax1.plot(self.trn_losses, label='train loss')
        ax1.plot(self.val_losses, label='test loss')
        ax2.plot(self.r2_scores, label='validation r2_scores', color='g')
        ax1.set_xlabel('epochs')
        ax1.set_ylabel('least square losses')
        ax2.set_ylabel('R2 Scores')
        ax2.legend()
        ax1.legend()
        plt.title('Losses and r2 scores for ' + name)
        if fname is not None:
            plt.savefig(imagedir + '/' + fname + '.jpg')
        plt.show()


def trn_validate(df, trn_end, val_end, layer_sizes):
    df_train = split_at(df, end_date=trn_end)
    df_val = split_at(df, start_date=trn_end, end_date=val_end)

    numeric_features_train, scaling_mean, scaling_std = scale(df_train)
    numeric_features_val, _, _ = scale(df_val, scaling_mean, scaling_std)
    nan_trn = numeric_features_train.isnull()
    nan_val = numeric_features_val.isnull()
    imputer = build_imputer(numeric_features_train)

    trn_imputed = apply_imputer(imputer, numeric_features_train)
    val_imputed = apply_imputer(imputer, numeric_features_val)

    trn_imputed = pd.concat((trn_imputed, nan_trn.astype(float)), 1)
    val_imputed = pd.concat((val_imputed, nan_val.astype(float)), 1)

    trn_sold, trn_Y = build_Y(df_train, trn_end)

    val_sold, val_Y = build_Y(df_val, dataset_end)

    n_features = trn_imputed.shape[1]

    model = Model(n_features, layer_sizes=layer_sizes)
    yhat = model.train(
        trn_imputed.values,
        val_imputed.values,
        trn_Y.values,
        val_Y.values,
        trn_sold.values,
        val_sold.values,
        epochs=10)
    model.visualize(
        'Loss and r2_scores ' + str(trn_end)[:10] + ' to \n' +
        str(val_end)[:10] + ' layers: ' + str(layer_sizes),
        str(trn_end)[:7] + ':' + str(val_end)[:7])
    return model.val_losses[-1], yhat, val_Y


def walkforward_optimization(df, FIRST_CUTOFF, LAST_CUTOFF, months_per_val=1):
    trn_end = FIRST_CUTOFF
    val_end = FIRST_CUTOFF + pd.DateOffset(months=months_per_val)
    validation_losses = pd.DataFrame()
    idx = 0
    yhat_all, val_Y_all = np.zeros(0), np.zeros(0)
    while val_end <= LAST_CUTOFF:
        idx += 1
        validation_loss, yhat, val_Y = trn_validate(df, trn_end, val_end,
                                                    [100, 100])
        yhat_all = np.concatenate((yhat_all, yhat.flatten()), 0)
        val_Y_all = np.concatenate((val_Y_all, val_Y), 0)
        validation_losses.loc[idx, 'cutoffs'] = val_end
        validation_losses.loc[idx, 'losses'] = validation_loss
        trn_end = trn_end + pd.DateOffset(months=months_per_val)
        val_end = val_end + pd.DateOffset(months=months_per_val)
    return validation_losses, yhat_all, val_Y_all


def visualize_walkforward_optimization_performance(validation_losses):
    plt.plot(validation_losses['cutoffs'], validation_losses['losses'])
    plt.xticks(rotation=-60)
    plt.legend()
    plt.xlabel('cutoff time')
    plt.ylabel('validation loss')
    plt.title('losses throughout walkforward optimization')

    plt.savefig(imagedir + '/walkforward.jpg')
    plt.show()


In [ ]:
validation_losses, yhat_all, val_Y_all = walkforward_optimization(df, FIRST_CUTOFF, LAST_CUTOFF)

In [ ]:
visualize_walkforward_optimization_performance(validation_losses)

In [ ]:
def plot_by_group(yhat_all, val_Y_all, increment=50, metric='abs'):

    # Either plot absolute error by group or directional error by group.
    if metric == 'abs':
        err = np.abs((yhat_all - val_Y_all) / val_Y_all)
        title = "Absolute Percent Errors by house sales group"
    else:
        err = (yhat_all - val_Y_all) / val_Y_all
        title = "Raw Percent Errors by house sales group"

    # group by distances of increment
    true_bin = ((val_Y_all // increment) + 1) * increment
    err_bins = {}
    
    # Place errors into appropriate bins
    for pe, tb in zip(err, true_bin):
        if not tb in err_bins.keys():
            err_bins[tb] = []
        err_bins[tb] += [pe * 100]

    # Sort keys (bins) and organize error lists
    err_list = [err_bins[key] for key in sorted(err_bins)]
    
    #Calculate means and Interquartile ranges
    medians = [np.mean(err_list[idx]) for idx in range(len(err_list))]
    q25s = [np.percentile(err_list[idx], 25) for idx in range(len(err_list))]
    q75s = [np.percentile(err_list[idx], 75) for idx in range(len(err_list))]
    
    keys = sorted(err_bins)
    
    # create blank graph and graph the axis variable
    _, ax = plt.subplots(figsize=(15, 5))

    # label axes
    ax.set(
        xlabel='true time to sell house',
        ylabel='IQR of percent error by group',
        title=title)

    # Add std deviation bars to the plot
    ax.errorbar(
        keys,
        medians,
        yerr=[[medians[i] - q25s[i] for i in range(len(medians))],
              [q75s[i] - medians[i] for i in range(len(medians))]],
        fmt='-o')
    
    # set where xticks should occur
    ax.set_xticks(keys)
    
    # Label the x-ticks
    xticks = [
        str(key - 50)[:-2] + ' to ' + str(key)[:-2] + '\n' + 'size group:' +
        str(len(err_list[idx])) for idx, key in enumerate(sorted(err_bins))
    ]
    ax.set_xticklabels(xticks)
    
    plt.show()

In [ ]:
plot_by_group(yhat_all, val_Y_all)
plot_by_group(yhat_all, val_Y_all, metric='raw')